In [3]:
import os
import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from collections import OrderedDict

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import SGD
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [18]:
FTRAIN = '../all/training.csv'
FTEST = '../all/test.csv'

SPECIALIST_SETTINGS = [
    dict(
        columns=(
            'left_eye_center_x', 'left_eye_center_y',
            'right_eye_center_x', 'right_eye_center_y',
            ),
        flip_indices=((0, 2), (1, 3)),
        ),

    dict(
        columns=(
            'nose_tip_x', 'nose_tip_y',
            ),
        flip_indices=(),
        ),

    dict(
        columns=(
            'mouth_left_corner_x', 'mouth_left_corner_y',
            'mouth_right_corner_x', 'mouth_right_corner_y',
            'mouth_center_top_lip_x', 'mouth_center_top_lip_y',
            ),
        flip_indices=((0, 2), (1, 3)),
        ),

    dict(
        columns=(
            'mouth_center_bottom_lip_x',
            'mouth_center_bottom_lip_y',
            ),
        flip_indices=(),
        ),

    dict(
        columns=(
            'left_eye_inner_corner_x', 'left_eye_inner_corner_y',
            'right_eye_inner_corner_x', 'right_eye_inner_corner_y',
            'left_eye_outer_corner_x', 'left_eye_outer_corner_y',
            'right_eye_outer_corner_x', 'right_eye_outer_corner_y',
            ),
        flip_indices=((0, 2), (1, 3), (4, 6), (5, 7)),
        ),

    dict(
        columns=(
            'left_eyebrow_inner_end_x', 'left_eyebrow_inner_end_y',
            'right_eyebrow_inner_end_x', 'right_eyebrow_inner_end_y',
            'left_eyebrow_outer_end_x', 'left_eyebrow_outer_end_y',
            'right_eyebrow_outer_end_x', 'right_eyebrow_outer_end_y',
            ),
        flip_indices=((0, 2), (1, 3), (4, 6), (5, 7)),
        ),
    ]

In [19]:
def load(test=False, cols=None):

    fname = FTEST if test else FTRAIN
    #테스트가 true면 test.csv 파일, false인경우 training.csv파일
    df = read_csv(os.path.expanduser(fname))
    """os.path모듈에서 제공되는 expanduser를 사용하여 절대 경로를 명시하여 줍니다.
    >>> expanduser('~\\devanix')
    'C:\\Documents and Settings\\Administrator\\devanix' """

    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))
    """Image열은 픽셀 값이 공백으로 구분이 되어 있기 때문에 apply 메서드를 이용하여 
    열에 대한 배열을 생성을 합니다."""
    if cols: # 열에 해당하는 값 가져오기
        df = df[list(cols) + ['Image']]

    print(df.count()) # 각 열의 값 수를 출력
    df = df.dropna() # NaN값에 대한 행 삭제

    X = np.vstack(df['Image'].values) / 255. 
    # 2개 이상의 배열을 합치기 위해 vstack 사용.
    # 픽셀 값 = 255 
    X = X.astype(np.float32)
    # float32로 타입 변환.
    if not test: # training.csv 파일에 대한 데이터 처리
        y = df[df.columns[:-1]].values # 15개 항목에 대한 데이터 값
        y = (y - 48) / 48 # 15개 항목에 대한 데이터 값을 -1 ~ 1사이의 값으로 변환
        X, y = shuffle(X, y, random_state=42) 
        #X, y에 대해 42개의 난수를 생성하여 셔플
        y = y.astype(np.float32)
    else:
        y = None

    return X, y


In [20]:
def load2d(test=False, cols=None):
    X, y = load(test, cols)
    X = X.reshape(-1, 1, 96, 96)
    return X, y

In [21]:
def plot_sample(x, y, axis):
    img = x.reshape(96, 96)
    axis.imshow(img, cmap='gray')
    axis.scatter(y[0::2]*48+48, y[1::2]*48+48, marker='x', s=10)

In [22]:
class FlippedImageDataGenerator(ImageDataGenerator):
    flip_indices = [
        (0, 2), (1, 3),
        (4, 8), (5, 9), (6, 10), (7, 11),
        (12, 16), (13, 17), (14, 18), (15, 19),
        (22, 24), (23, 25),
        ] #반전 된 이미지에서 keypoint의 위치 변경

    def next(self):
        X_batch, y_batch = super(FlippedImageDataGenerator, self).next()
        batch_size = X_batch.shape[0]
        indices = np.random.choice(batch_size, batch_size/2, replace=False)
        X_batch[indices] = X_batch[indices, :, :, ::-1]

        if y_batch is not None: # x좌표 변경
            y_batch[indices, ::2] = y_batch[indices, ::2] * -1

            for a, b in self.flip_indices: # left_eye_center_x -> right_eye_center_x
                y_batch[indices, a], y_batch[indices, b] = (
                    y_batch[indices, b], y_batch[indices, a]
                )

        return X_batch, y_batch
    """ 반전된 이미지를 training 데이터 X에 단순히 추가하는 방식이 아닌 랜덤으로 반전시킴으로서 메모리 소비를
    줄 일수 있다. """

In [23]:
model = Sequential() # 객체 생성

model.add(Convolution2D(32, (3, 3), input_shape=(1, 96, 96), data_format='channels_first'))
# 텐서의 출력 생성
# Convolution2D(32, (3, 3) -> 텐서의 출력 개수
# input_shape=(1,96,96) -> 96x96 의 그림 1은 보폭
# data_format='channels_first' -> 모양 있는 입력
model.add(Activation('relu')) #relu - max(0,x) 처럼 음수에 대해서만 0으로 처리하는 함수


model.add(MaxPooling2D(pool_size=(2, 2))) # 시간 데이터에 대한 최대 출링 연상 최대 풀링 창의 크기 2x2
model.add(Dropout(0.1)) # 무작위 항목을 설정 

model.add(Convolution2D(64, 2, 2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Convolution2D(128, 2, 2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(1000)) # 노드의 개수
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dense(30))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2))`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (2, 2))`
  del sys.path[0]


In [43]:
def fit_specialists(fname_pretrain=None):
    specialists = OrderedDict() #순서대로 저장하는 딕셔너리
    start = 0.03
    stop = 0.001
    nb_epoch = 500 # 학습 epoch(학습 시 한 번에 처리할 블록 길이) 횟수

    for setting in SPECIALIST_SETTINGS:

        cols = setting['columns']
        X, y = load2d(cols=cols)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        model_specialist = model_from_json(model.to_json()) # 모델의 JSON 문자열 반환

        if fname_pretrain:
            model_specialist.load_weights(fname_pretrain) # 생성 된 HDF5 파일에서의 모델 가중치 로드

        model_specialist.layers.pop()
        model_specialist.outputs = [model_specialist.layers[-1].output] # 출력 텐서 목록
        model_specialist.layers[-1].outbound_nodes = [] #레이어의 병합 목록
        model_specialist.add(Dense(len(cols)))

        # 최적화
        sgd = SGD(lr=start, momentum=0.9, nesterov=True) 
        """SGD 운동량, 학습 속도 감소에 대한 지원
        lr = 학습속도, momentum = SGD 가속"""
        model_specialist.compile(loss='mean_squared_error', optimizer=sgd)
        # SGD 인스턴스화 , 기본 매개변수로 사용.

        # flip_indices 튜플 정의
        flipgen = FlippedImageDataGenerator()
        flipgen.flip_indices = setting['flip_indices']

        # 조기 종료
        """이전 epoch와 비교해서 오차가 증가하면 학습을 중단하는 부분"""
        early_stop = EarlyStopping(patience=100) #patience - 개선이 없는 epoch 100번 지속되면 학습 종료
        learning_rates = np.linspace(start, stop, nb_epoch) # 일정 간격마다 숫자를 반환
        change_lr = LearningRateScheduler(lambda epoch: float(learning_rates[epoch])) # 시간 반환

        print("Training model : {} ,epochs : {} ".format(cols, nb_epoch))

        #데이터 모델 트레이닝
        hist = model_specialist.fit_generator(flipgen.flow(X_train, y_train), 
                                     #ImageDataGenerator.flow를 통한 학습 데이터의 증가
                                     samples_per_epoch=X_train.shape[0],
                                     # 완성 된 epoch에 다음 epoch에 대한 단계의 수
                                     nb_epoch=nb_epoch, #학습 개수 : 500
                                     validation_data=(X_val, y_val), # loss, val_loss의 모델 평가
                                     callbacks=[change_lr, early_stop]) # 조기종료

        specialists[cols] = model_specialist

In [ ]:
fit_specialists()

left_eye_center_x     7039
left_eye_center_y     7039
right_eye_center_x    7036
right_eye_center_y    7036
Image                 7049
dtype: int64
Training model for columns ('left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x', 'right_eye_center_y') for 500 epochs


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., callbacks=[<keras.ca..., steps_per_epoch=175, epochs=500)`


Epoch 1/500
175/175 [==============================] - 287s 2s/step - loss: 0.0060 - val_loss: 0.0051
Epoch 2/500
175/175 [==============================] - 287s 2s/step - loss: 0.0042 - val_loss: 0.0051
Epoch 3/500
175/175 [==============================] - 282s 2s/step - loss: 0.0043 - val_loss: 0.0051
Epoch 4/500
175/175 [==============================] - 280s 2s/step - loss: 0.0042 - val_loss: 0.0051
Epoch 5/500
175/175 [==============================] - 282s 2s/step - loss: 0.0042 - val_loss: 0.0051
Epoch 6/500
175/175 [==============================] - 280s 2s/step - loss: 0.0042 - val_loss: 0.0051
Epoch 7/500
175/175 [==============================] - 281s 2s/step - loss: 0.0041 - val_loss: 0.0051
Epoch 8/500
175/175 [==============================] - 281s 2s/step - loss: 0.0043 - val_loss: 0.0051
Epoch 9/500
175/175 [==============================] - 77301s 442s/step - loss: 0.0043 - val_loss: 0.0051
Epoch 10/500
175/175 [==============================] - 286s 2s/step - loss: 0

175/175 [==============================] - 282s 2s/step - loss: 0.0028 - val_loss: 0.0037
Epoch 81/500
175/175 [==============================] - 283s 2s/step - loss: 0.0028 - val_loss: 0.0038
Epoch 82/500
175/175 [==============================] - 284s 2s/step - loss: 0.0031 - val_loss: 0.0037
Epoch 83/500
175/175 [==============================] - 281s 2s/step - loss: 0.0029 - val_loss: 0.0036
Epoch 84/500
175/175 [==============================] - 281s 2s/step - loss: 0.0025 - val_loss: 0.0036
Epoch 85/500
175/175 [==============================] - 282s 2s/step - loss: 0.0030 - val_loss: 0.0037
Epoch 86/500
175/175 [==============================] - 281s 2s/step - loss: 0.0030 - val_loss: 0.0036
Epoch 87/500
175/175 [==============================] - 285s 2s/step - loss: 0.0025 - val_loss: 0.0036
Epoch 88/500
175/175 [==============================] - 281s 2s/step - loss: 0.0029 - val_loss: 0.0037
Epoch 89/500
175/175 [==============================] - 283s 2s/step - loss: 0.0029 - 

175/175 [==============================] - 282s 2s/step - loss: 0.0020 - val_loss: 0.0030
Epoch 160/500
175/175 [==============================] - 281s 2s/step - loss: 0.0022 - val_loss: 0.0030
Epoch 161/500
175/175 [==============================] - 280s 2s/step - loss: 0.0021 - val_loss: 0.0030
Epoch 162/500
175/175 [==============================] - 285s 2s/step - loss: 0.0020 - val_loss: 0.0030
Epoch 163/500
175/175 [==============================] - 281s 2s/step - loss: 0.0021 - val_loss: 0.0030
Epoch 164/500
175/175 [==============================] - 282s 2s/step - loss: 0.0022 - val_loss: 0.0030
Epoch 165/500
175/175 [==============================] - 287s 2s/step - loss: 0.0021 - val_loss: 0.0030
Epoch 166/500
175/175 [==============================] - 290s 2s/step - loss: 0.0021 - val_loss: 0.0030
Epoch 167/500
175/175 [==============================] - 289s 2s/step - loss: 0.0022 - val_loss: 0.0029
Epoch 168/500
175/175 [==============================] - 292s 2s/step - loss: 

Epoch 238/500
175/175 [==============================] - 283s 2s/step - loss: 0.0018 - val_loss: 0.0028
Epoch 239/500
175/175 [==============================] - 282s 2s/step - loss: 0.0017 - val_loss: 0.0028
Epoch 240/500
175/175 [==============================] - 281s 2s/step - loss: 0.0019 - val_loss: 0.0028
Epoch 241/500
175/175 [==============================] - 280s 2s/step - loss: 0.0017 - val_loss: 0.0028
Epoch 242/500
175/175 [==============================] - 282s 2s/step - loss: 0.0020 - val_loss: 0.0028
Epoch 243/500
175/175 [==============================] - 279s 2s/step - loss: 0.0019 - val_loss: 0.0028
Epoch 244/500
175/175 [==============================] - 281s 2s/step - loss: 0.0018 - val_loss: 0.0029
Epoch 245/500
175/175 [==============================] - 282s 2s/step - loss: 0.0018 - val_loss: 0.0028
Epoch 246/500
175/175 [==============================] - 283s 2s/step - loss: 0.0019 - val_loss: 0.0028
Epoch 247/500
175/175 [==============================] - 281s 2s

175/175 [==============================] - 288s 2s/step - loss: 0.0016 - val_loss: 0.0028
Epoch 317/500
175/175 [==============================] - 288s 2s/step - loss: 0.0018 - val_loss: 0.0029
Epoch 318/500
175/175 [==============================] - 283s 2s/step - loss: 0.0016 - val_loss: 0.0029
Epoch 319/500
175/175 [==============================] - 284s 2s/step - loss: 0.0016 - val_loss: 0.0028
Epoch 320/500
175/175 [==============================] - 285s 2s/step - loss: 0.0016 - val_loss: 0.0029
Epoch 321/500
175/175 [==============================] - 281s 2s/step - loss: 0.0016 - val_loss: 0.0029
Epoch 322/500
175/175 [==============================] - 285s 2s/step - loss: 0.0017 - val_loss: 0.0029
Epoch 323/500
175/175 [==============================] - 285s 2s/step - loss: 0.0016 - val_loss: 0.0028
Epoch 324/500
175/175 [==============================] - 285s 2s/step - loss: 0.0017 - val_loss: 0.0028
Epoch 325/500
175/175 [==============================] - 283s 2s/step - loss: 